In [3]:
import numpy as np
from scipy.signal import butter, filtfilt
from scipy.io import wavfile
from IPython.display import Audio, display

# -------------------- Functions --------------------
def adc(signal, fs, delta):
    reconstructed = 0
    bitstream = []
    for sample in signal:
        error = sample - reconstructed
        bit = 1 if error >= 0 else 0
        reconstructed += delta if bit else -delta
        bitstream.append(bit)
    return np.array(bitstream)

def linecoder(bitstream, r, A, Fs_sim):
    samples_per_symbol = int(Fs_sim / r)
    return np.repeat(A * np.array(bitstream), samples_per_symbol)

def askmodulator(nrz, fc, Fs_sim):
    t = np.arange(len(nrz)) / Fs_sim
    return nrz * np.cos(2 * np.pi * fc * t)

def channel(signal, N0, B):
    noise = np.random.normal(0, np.sqrt(N0 * B), len(signal))
    return signal + noise

def askdemodulator(received, fc, Fs_sim, cutoff):
    t = np.arange(len(received)) / Fs_sim
    demod = received * np.cos(2 * np.pi * fc * t)
    b, a = butter(5, cutoff / (0.5 * Fs_sim), btype='low')
    return filtfilt(b, a, demod)

def linedecoder(filtered, r, Fs_sim, A):
    samples_per_symbol = int(Fs_sim / r)
    sampled = filtered[samples_per_symbol//2::samples_per_symbol]
    return (sampled > A/2).astype(int)

def dac(bitstream, delta, fs):
    reconstructed = np.cumsum(delta * (2 * np.array(bitstream) - 1))
    return reconstructed

# -------------------- Main Script --------------------
if __name__ == "__main__":
    # ========== Parameters ==========
    fs = 8000          # Standard audio sampling rate (Hz)
    delta = 0.01        # Quantization step
    r = 1000            # Baud rate (symbols/sec)
    A = 1.0             # NRZ amplitude
    fc = 4000          # Carrier frequency (< fs/2 to avoid aliasing)
    B = 1000            # Channel bandwidth
    N0 = 0.01          # Start with low noise
    input_file = 'audio.wav'

    # ========== Signal Processing ==========
    # Read and normalize input
    Fs_audio, audio = wavfile.read(input_file)
    audio = audio.astype(np.float32) / np.max(np.abs(audio))

    # Resample if needed (code omitted for brevity)

    # ADC (Delta Modulation)
    bits = adc(audio, fs, delta)

    # Line Coding
    nrz = linecoder(bits, r, A, fs)  # Use fs for simulation rate

    # ASK Modulation
    modulated = askmodulator(nrz, fc, fs)

    # Channel
    noisy = channel(modulated, N0, B)

    # ASK Demodulation
    demodulated = askdemodulator(noisy, fc, fs, cutoff=2*r)

    # Line Decoding
    decoded_bits = linedecoder(demodulated, r, fs, A)

    # DAC (Delta Demodulation)
    output_signal = dac(decoded_bits, delta, fs)

    # ========== Audio Output ==========
    # Normalize and clip
    output_signal = output_signal.astype(np.float32)
    output_signal /= np.max(np.abs(output_signal) + 1e-6)
    output_signal = np.clip(output_signal, -1.0, 1.0)

    display(Audio(output_signal, rate=fs, autoplay=True))

<ipython-input-3-9ef109b4b416>:58: WavFileWarning: Chunk (non-data) not understood, skipping it.
  Fs_audio, audio = wavfile.read(input_file)
